In [3]:
import boto3
import pendulum

In [5]:
s3 = boto3.resource('s3')
private_bucket = s3.Bucket('chn-ghost-buses-private')
public_bucket = s3.Bucket('chn-ghost-buses-public')

In [19]:
# copy raw data from the private bucket to the public bucket 
start_date = '2022-08-10'
end_date = '2022-10-08'

date_range = [d for d in pendulum.period(pendulum.from_format(start_date, 'YYYY-MM-DD'), pendulum.from_format(end_date, 'YYYY-MM-DD')).range('days')]

In [18]:
for day in date_range:
    date_str = day.to_date_string()
    print(f"Processing {date_str} at {pendulum.now().to_datetime_string()}")
    objects = private_bucket.objects.filter(Prefix = f'bus_data/{date_str}')
    for obj in objects:
        public_bucket.copy({'Bucket': private_bucket.name, 'Key': obj.key}, obj.key)

Processing 2022-05-19 at 2022-10-10 11:22:23
Processing 2022-05-20 at 2022-10-10 11:22:28
Processing 2022-05-21 at 2022-10-10 11:23:36
Processing 2022-05-22 at 2022-10-10 11:24:37
Processing 2022-05-23 at 2022-10-10 11:25:35
Processing 2022-05-24 at 2022-10-10 11:26:41
Processing 2022-05-25 at 2022-10-10 11:27:48
Processing 2022-05-26 at 2022-10-10 11:28:56
Processing 2022-05-27 at 2022-10-10 11:30:06
Processing 2022-05-28 at 2022-10-10 11:31:15
Processing 2022-05-29 at 2022-10-10 11:32:19
Processing 2022-05-30 at 2022-10-10 11:33:18
Processing 2022-05-31 at 2022-10-10 11:34:17
Processing 2022-06-01 at 2022-10-10 11:35:24
Processing 2022-06-02 at 2022-10-10 11:36:31
Processing 2022-06-03 at 2022-10-10 11:37:36
Processing 2022-06-04 at 2022-10-10 11:38:41
Processing 2022-06-05 at 2022-10-10 11:39:42
Processing 2022-06-06 at 2022-10-10 11:40:41
Processing 2022-06-07 at 2022-10-10 11:41:47
Processing 2022-06-08 at 2022-10-10 11:42:51
Processing 2022-06-09 at 2022-10-10 11:43:56
Processing

In [21]:
for day in date_range:
    date_str = day.to_date_string()
    print(f"Processing {date_str} at {pendulum.now().to_datetime_string()}")
    data_objects = private_bucket.objects.filter(Prefix = f'bus_full_day_data_v2/{date_str}.csv')
    error_objects = private_bucket.objects.filter(Prefix = f'bus_full_day_errors_v2/{date_str}.csv')
    for obj in data_objects:
        public_bucket.copy({'Bucket': private_bucket.name, 'Key': obj.key}, obj.key)
    for obj in error_objects: 
        public_bucket.copy({'Bucket': private_bucket.name, 'Key': obj.key}, obj.key)

Processing 2022-08-10 at 2022-10-10 17:01:41
Processing 2022-08-11 at 2022-10-10 17:01:43
Processing 2022-08-12 at 2022-10-10 17:01:44
Processing 2022-08-13 at 2022-10-10 17:01:45
Processing 2022-08-14 at 2022-10-10 17:01:46
Processing 2022-08-15 at 2022-10-10 17:01:47
Processing 2022-08-16 at 2022-10-10 17:01:48
Processing 2022-08-17 at 2022-10-10 17:01:49
Processing 2022-08-18 at 2022-10-10 17:01:50
Processing 2022-08-19 at 2022-10-10 17:01:51
Processing 2022-08-20 at 2022-10-10 17:01:52
Processing 2022-08-21 at 2022-10-10 17:01:53
Processing 2022-08-22 at 2022-10-10 17:01:54
Processing 2022-08-23 at 2022-10-10 17:01:55
Processing 2022-08-24 at 2022-10-10 17:01:57
Processing 2022-08-25 at 2022-10-10 17:01:57
Processing 2022-08-26 at 2022-10-10 17:01:58
Processing 2022-08-27 at 2022-10-10 17:01:59
Processing 2022-08-28 at 2022-10-10 17:02:01
Processing 2022-08-29 at 2022-10-10 17:02:03
Processing 2022-08-30 at 2022-10-10 17:02:04
Processing 2022-08-31 at 2022-10-10 17:02:05
Processing